In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException, TimeoutException
import time
import requests
import os 

In [25]:
directorio_enlaces = '/Users/carlosabraham1/Desktop/autogenrated_databse/codigos/enlaces_rsc'

In [26]:
if not os.path.exists(directorio_enlaces):
    os.makedirs(directorio_enlaces)

In [5]:
web = 'https://pubs.rsc.org'
path = '/Users/carlosabraham1/Documentos1/ciencia_de_datos/chromedriver_mac_arm64/chromedriver'

In [6]:
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory":directorio_enlaces,
    "download.prompt_for_download":False,
    "download.directory.upgrade":True,
    "safebrowsing.enabled":True,
    "profile.default_content_settings.popups":0,
    "profile.content_settings.exceptions.automatic_downloads.*.setting":1
}

In [7]:
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
driver.get(web) 

In [8]:
def manejar_alerta():
    try:
        # Esperar un momento para que la alerta aparezca
        WebDriverWait(driver, 10).until(EC.alert_is_present())
        alert = driver.switch_to.alert
        alert.accept()  # Aceptar la alerta
        print("Alerta aceptada.")
    except TimeoutException:
        print("No apareció ninguna alerta en el tiempo de espera.")    
    except (NoAlertPresentException, UnexpectedAlertPresentException):
        print("No hay alerta presente o no se pudo manejar.")


In [9]:
def busqueda_articulo (termino):
    boton_buscar = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="SearchText"]'))
    )
    boton_buscar.send_keys(termino)
    boton_buscar.send_keys(Keys.RETURN)
    time.sleep(2)

In [10]:
def filtrar_articulo ():
    boton_open_access = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH,'//*[@id="filterPanelArticles"]/div[2]/div[2]/div/ul/li/a/span'))
    )
    boton_open_access.click()

In [11]:
def obtener_enlaces (): 
    try:
        # Espera hasta que se localicen los enlaces
        enlaces_html = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.btn.btn--tiny:not(.btn--primary)'))
        )
        return enlaces_html
    except TimeoutException:
        print("Error: No se encontraron enlaces en esta página.")
        return []

In [12]:
def navegar_siguiente_pagina():
    try:
        siguiente_pagina = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="tabArticles"]/div[2]/div[27]/nav/ul/li[2]/a/img'))
        )
        siguiente_pagina.click()
        time.sleep(2)
    except TimeoutException:
        print('no hay mas paginas')
        raise


In [13]:
busqueda_articulo("doped CeO2 band gap")
manejar_alerta()
filtrar_articulo()
todos_los_enlaces = []
for i in range(10):
    print(f'pagina{i+1}')
    enlaces_html = obtener_enlaces()
    if enlaces_html:
        todos_los_enlaces.extend(enlaces_html)
    else: 
        print("no hay mas paginas") 
    try: 
        navegar_siguiente_pagina()
    except TimeoutException:
        print("algo fallo")
        break    
print(f"enalces extraidos: {len(todos_los_enlaces)}")                

No apareció ninguna alerta en el tiempo de espera.
pagina1
pagina2
pagina3
pagina4
pagina5
pagina6
pagina7
pagina8
pagina9
pagina10
enalces extraidos: 259


In [14]:
lista_href = []

In [15]:
if todos_los_enlaces:
    for enlace in todos_los_enlaces:
        try:
            href = enlace.get_attribute("href")
            if href:
                lista_href.append(href)
            else:
                print(f"el elemento {enlace} no tiene un atributo href valido")
        except Exception as e:
            print(f'El error al obtener el atributo href :{e}')
        else:
            print('No se encontraron los enlaces en la lista')               

El error al obtener el atributo href :Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=130.0.6723.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000100f5f634 cxxbridge1$str$ptr + 3645404
1   chromedriver                        0x0000000100f57e94 cxxbridge1$str$ptr + 3614780
2   chromedriver                        0x00000001009c4104 cxxbridge1$string$len + 88416
3   chromedriver                        0x00000001009c919c cxxbridge1$string$len + 109048
4   chromedriver                        0x00000001009cab0c cxxbridge1$string$len + 115560
5   chromedriver                        0x0000000100a40808 cxxbridge1$string$len + 598116
6   chromedriver                        0x0000000100a3fbd0 cxxbridge1$string$len + 594988
7   chromedriver                        0x00000001009

In [16]:
try:
    for i in range(3):  # Cambia el rango según cuántas páginas quieras recorrer
        # Re-localiza los enlaces en cada iteración para evitar el error de "stale element"
        enlaces_html = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.btn.btn--tiny:not(.btn--primary)'))
        )

        for enlace in enlaces_html:
            try:
                # Reintentar la localización de cada enlace antes de acceder a "href"
                href = enlace.get_attribute("href")
                if href and href not in lista_href:  # Evitar duplicados
                    lista_href.append(href)
            except Exception as e:
                print(f"Error al obtener el atributo href: {e}")
        
        # Ir a la siguiente página (cambia según la estructura de tu sitio)
        siguiente_pagina = driver.find_element(By.XPATH, '//*[@id="tabArticles"]/div[2]/div[27]/nav/ul/li[2]/a/img')
        siguiente_pagina.click()
        time.sleep(2)  # Asegúrate de dar tiempo a la página para cargar correctamente

except Exception as e:
    print(f"Error al procesar los enlaces o navegar por las páginas: {e}")

# Imprime la lista de enlaces extraídos
print("Enlaces encontrados:")
for href in lista_href:
    print(href)

Enlaces encontrados:
https://pubs.rsc.org/en/results?searchtext=doped+CeO2+band+gap&SortBy=Relevance&PageSize=25&tab=journal&fcategory=journal&filter=journal&Article%20Access=Open+Access#
https://pubs.rsc.org/en/results?searchtext=doped+CeO2+band+gap&SortBy=Relevance&PageSize=25&tab=journal&fcategory=journal&filter=journal&Article%20Access=Open+Access#
https://pubs.rsc.org/en/results?searchtext=doped+CeO2+band+gap&SortBy=Relevance&PageSize=25&tab=journal&fcategory=journal&filter=journal&Article%20Access=Open+Access#
https://pubs.rsc.org/en/results?searchtext=doped+CeO2+band+gap&SortBy=Relevance&PageSize=25&tab=journal&fcategory=journal&filter=journal&Article%20Access=Open+Access#
https://pubs.rsc.org/en/results?searchtext=doped+CeO2+band+gap&SortBy=Relevance&PageSize=25&tab=journal&fcategory=journal&filter=journal&Article%20Access=Open+Access#
https://pubs.rsc.org/en/results?searchtext=doped+CeO2+band+gap&SortBy=Relevance&PageSize=25&tab=journal&fcategory=journal&filter=journal&Article

In [28]:
archivos_enlaces = os.path.join(directorio_enlaces, 'enlaces obtenidos.txt')



In [30]:
try:
    print(f"Guardando archivo en: {archivos_enlaces}")  # Imprime la ruta completa del archivo
    with open(archivos_enlaces, 'w', encoding='utf-8') as archivo:
        for href in lista_href:
            archivo.write(href + '\n')
    print(f"Enlaces guardados exitosamente en {archivos_enlaces}")
except Exception as e:
    print(f"Error al guardar los enlaces: {e}")


Guardando archivo en: /Users/carlosabraham1/Desktop/autogenrated_databse/codigos/enlaces_rsc/enlaces obtenidos.txt
Enlaces guardados exitosamente en /Users/carlosabraham1/Desktop/autogenrated_databse/codigos/enlaces_rsc/enlaces obtenidos.txt


In [29]:
# try:
#     with open(archivos_enlaces, 'r', encoding='utf-8') as archivo: 
#         for herf in lista_href:
#             archivo.write(href + '\n')
#         print(f'enlaces guardados con exito en {archivos_enlaces}') 

# except Exception as e:
#     print(f'Error al guardar los enlaces: {e}')

Error al guardar los enlaces: [Errno 2] No such file or directory: '/Users/carlosabraham1/Desktop/autogenrated_databse/codigos/enlaces_rsc/enlaces obtenidos.txt'
